### Форматирование в С++

В лекции:
* С-шный форматированный вывод
* Потоки в С++
* `boost::format`
* fmtlib
* `std::format` в C++20
* сравнение билиотек

<br />

##### naive C

https://en.cppreference.com/w/cpp/io/c

https://en.cppreference.com/w/cpp/io/c/fprintf

В чистом С для форматирования есть семейство функций `printf`.

Они поддерживают набор форматов для встроенных типов:
* целые числа
* вещественные числа
* С-строки
* указатели

```c++
printf("%i < %s < %.2f", 3, "pi", 3.15f);
```

**Вопрос**: какие преимущества и недостатки у этого способа?

<details>
<summary>преимущества</summary>
<p>

* быстрое исполнение (сравнительно)
* быстрая скорость компиляции
* маленький размер бинарного кода
* мало зависимостей (требуется только libc)

</p>
</details>

<details>
<summary>недостатки</summary>
<p>

* обязанность программиста следить за соответствием типов и форматов (много ошибок)

</p>
</details>

<br />

C-шный вариант позволяет записывать результат не только в `FILE*`, но и в строковый буфер:

```c++
char buf[256];
std::sprintf(buf, "%i < %s < %.2f", 3, "pi", 3.15f);
```

Это весьма удобный С-шный способ формировать форматированные строки.

**Вопрос**: какой недостаток вы видите в этом способе?

<details>
<summary>ответ</summary>
<p>
    
Функция `sprintf` не может проконтроллировать выход за границы буфера, программист обязан умозрительно гарантировать, что буфера хватит. Конечно, программисты иногда ошибаются.
    
</p>
</details>

<br />

Чтобы справиться с проблемой, в С++11 добавили `snprintf` - форматированный вывод в строку с контролем длины:

```c++
// форматируем строку с указанием размера буфера,
// std::snprintf проконтроллирует выход за границы массива
char buf[256];
int rv = std::snprintf(buf, 256, "%i < %s < %.2f", 3, "pi", 3.15f);
// теперь надо разобрать код возврата

// отрицательные значения - индикатор ошибки внутри snprintf
if (rv < 0)
    std::cout << "formatting error" << std::endl;

// значения < 256 - значит null-terminated output успешно записано в буфер
if (rv < 256)
    std::cout << "succeed, symbols written: " << rv << std::endl;

// значение >= 256 - сколько символов без нуля нужно в буфере,
// чтобы полностью записать строку
if (rv >= 256)
    std::cout << "buffer is too small, required size is: " << rv << std::endl;
```

Трюк с предварительным определением нужного размера:

```c++
// оценим, сколько символов требуется, чтобы записать форматированную строку
const int sz = std::snprintf(nullptr, 0, "sqrt(2) = %f", std::sqrt(2));

// выделим на куче нужное кол-во символов +1 для нуль-терминатора
std::vector<char> buf(sz + 1);

// собственно, само форматирование
std::snprintf(&buf[0], buf.size(), "sqrt(2) = %f", std::sqrt(2));
```

Такой подход безопаснее, но, увы, требуется дважды выполнить полное форматирование строки:
* сначала чтобы узнать размер
* затем чтобы записать результат

<br />

##### C++ streams

https://en.cppreference.com/w/cpp/io/manip

Стандартные С++ потоки позволяют задавать форматирование:

Форматирование `bool`:
    
```c++
std::cout << std::boolalpha 
          << "true: " << true << std::endl
          << "false: " << false << std::endl;
std::cout << std::noboolalpha 
          << "true: " << true << std::endl
          << "false: " << false << std::endl;
```

Вывод:

```sh
true: true
false: false
true: 1
false: 0
```

Основа системы счисления для `oct/hex` чисел:
    
```c++
std::cout << std::hex
          << "42 = " << std::showbase << 42 << std::endl
          << "42 = " << std::noshowbase << 42 << std::endl;
```

Вывод:

```sh
42 = 0x2a
42 = 2a
```

Форматирование `float`:

```c++
const long double pi = std::acos(-1.L);
std::cout << "default precision (6): " << pi << std::endl
          << "std::setprecision(10): " << std::setprecision(10) << pi << std::endl;

std::cout << "fixed:      " << std::fixed        << 0.01 << std::endl
          << "scientific: " << std::scientific   << 0.01 << std::endl
          << "hexfloat:   " << std::hexfloat     << 0.01 << std::endl
          << "default:    " << std::defaultfloat << 0.01 << std::endl;
```

Вывод:

```sh
default precision (6): 3.14159
std::setprecision(10): 3.141592654

fixed:      0.010000
scientific: 1.000000e-02
hexfloat:   0x1.47ae147ae147bp-7
default:    0.01
```

И некоторые другие способы отформатировать значения, смотрите их в документации по ссылке.

А лучше не смотрите, т.к. далее мы рассмотрим более продвинутые способы.

<br />

##### boost::format

https://www.boost.org/doc/libs/1_66_0/libs/format/doc/format.html

Способы форматирования, поддерживаемые `boost::format`:

* `%spec`, spec - [printf specification](https://www.boost.org/doc/libs/1_66_0/libs/format/doc/format.html#printf_directives)

boost::format позволяет эмулировать обычный сишный `printf` [за небольшим числом хитрых исключений](https://www.boost.org/doc/libs/1_66_0/libs/format/doc/format.html#printf_differences)

```c++
std::cout << boost::format("%i <= pi <= %f") % 3 % 3.15 << std::endl;
```

* `%|spec|`, spec - [printf specification](https://www.boost.org/doc/libs/1_66_0/libs/format/doc/format.html#printf_directives)

У этого способа есть две особенности:
    * авторы boost утверждают, что он улучшает читабельность (возможно, это так)
    * не требуется указание типа:
  
```c++
// все результаты ниже имеют ширину 5 и выровнены влево
boost::format("%|-5|") % 3.14;
boost::format("%|-5|") % 3;
boost::format("%|-5|") % "3.1"s;
```
  
* `%N%` - позиционные аргументы

```c++
boost::format("%1% <= pi <= %2%") % 3 % 3.14;
```

<br />

##### поддержка пользовательских типов

Чтобы вывести пользоветальский тип, достаточно определить для него `operator<<`, здесь всё естественно:

```c++
struct Point
{
    float x;
    float y;
};

std::ostream& operator << (std::ostream& os, const Point& p)
{
    return os << '(' << p.x << ',' << p.y << ')';
}

Point p{3, 4};
std::cout << boost::format("Point of interest is %1%") % p;
```

<br />

##### что такое `boost::format`?

`boost::format` - это объект-`formatter`:

```c++
// создали объект-formatter
boost::format f("%1% %2% %3% %1%");

// скормили объекту аргументы
f % 10 % 20 % 30; 

// после того как formatter насытили аргументами,
// можно спрашивать с него результат, например, через <<
std::cout << f;  // "10 20 30 10"
    
// можно спрашивать с него результат несколько раз
std::cout << f;  // "10 20 30 10"
std::string s = f.str();

// можно начать его насыщать аргументами заново
f % 1001;
try
{
    std::cout << f;
}
catch (const boost::io::too_few_args&)
{
    std::cout << "Formatter is not fed" << std::endl;
}

// насытив formatter, можно снова спрашивать с него результат
std::cout << f % "abc" % "def";

// можно через методы объекта модифицировать спецификацию формата
f = boost::format("%1% %2% %3% %2% %1%");
f.modify_item(4, boost::io::group(std::setfill('_'),
                                  std::hex,
                                  std::showbase,
                                  std::setw(5)));
std::cout << f % 1 % 2 % 3; // "1 2 3 __0x2 1 \n"

// задавать нумерованные аргументы по номеру явно
f = boost::format("%1% %2% %3% %2% %1%");
f.bind_arg(1, "x");
f.bind_arg(2, "y");
f.bind_arg(3, "z");
std::cout << f;

// поведение аргументов, заданных через bind и %, различно:
//   * через bind - привязанные (bounded) аргументы
//   * через %    - регулярные (regular) аргументы
f = boost::format("%1% %2% %3% %2% %1%");
f.bind_arg(1, "10");
f.bind_arg(1, "11");  // перезадали аргумент по номеру
std::cout << f % 2 % 3;  // "11 2 3 2 11"

try  
{
    // бросит исключение, потому что аргумент N1 привязанный
    // значит через % насыщаются только "регулярные" N2 и N3
    std::cout << f % 6 % 7 % 8;
}
catch (const boost::io::too_many_args&) 
{
    std::cout << "Formatter is fed out";
}

// очистка регулярных аргументов
f.clear();

// очитска и регулярных, и связанных аргументов
f.clear_binds();
```

<br />

`boost::format` - богатый и удобный инструмент. Код с его использованием становится короче, понятнее и читабельнее.

Сравните:

```c++
std::cout << "ERROR: failed to open " << filename << " at line " << line_number << ". OS response: " << reponse << '\n';

std::cout << boost::format("ERROR: failed to open %1% at line %2%. OS response: %3%\n")
             % filename % line_number % response;
```

Но `boost::format` обладает двумя сущестенными недостатками:
* долгая компиляция
* медленный

(цифры ниже)

<br />

##### fmtlib

https://fmt.dev/latest/

https://github.com/fmtlib/fmt

Если вы всегда хотели так же элегантно форматировать строки как это сделано в python через метод str.format, то будущее уже здесь.

```c++
// format to string
std::string s = fmt::format("{} was {} when he became an epic hero", "Ilya", 33);

// format to local buffer
char[16] out = "";
fmt::format_to(out, "{}", 42);

// format to local buffer with restrictions
char[16] out = "";
fmt::format_to_n(out, 16, "{}", 42);

// format to output iterator
std::vector<char> out;
fmt::format_to(std::back_inserter(out), "{}", 42);

// format to std out
fmt::print("{} was {} when he became an epic hero", "Ilya", 33);

// format to FILE* stream
FILE *f = ...;
fmt::print(f, "{} was {} when he became an epic hero", "Ilya", 33);
```

<br />

Аналогично python-овскому `str.format`:

* форматирование аргументов

 ```c++
fmt::print('pi is approximately equal to {:.3f}', M_PI);
```

* нумерация аргументов

```c++
// заметьте, что в отличие от |boost::format| аргументы нумеруются с 0
fmt::print("{0} was at war with {1} at least 12 times. {0} has won approximately 7 times.",
           "Russian Empire",
           "Ottomans Empire"s);
```

* именование аргументов
    
```c++
fmt::print("{ru} was at war with {ot} at least 12 times. {ru} has won approximately 7 times.",
           fmt::arg("ru", "Russian Empire"),
           fmt::arg("ot", "Ottomans Empire"s));
```

<br />

##### compile-time проверки формата

Но у нас есть С++, и в отличие от python, мы можем проверить корректность строки форматирования на этапе компиляции:

```c++
// compile-time error: 'd' is an invalid specifier for strings.
std::string s = format(FMT_STRING("{:d}"), "foo");

// compile-time error: argument N2 is not set
std::string s = format(FMT_STRING("{2}"), 42);
```

<br />

##### Форматирование пользовательских типов

```c++
// Пользовательский тип
struct Point
{
    float x;
    float y;
    float z;
};

// Специализация шаблона fmt::formatter для пользовательского типа.
//
// Она определяет:
//   * какие опции форматирования доступны
//   * как их обрабатывать
//
// В примере рассмотрим опции:
//   {}   - формат по умолчанию
//   {:f} - формат с фиксированной точкой
//   {:e} - научный формат
//
// Нужно определить 2 метода:
//   |parse| - нужно разобрать строку формата
//             и сохранить результат как внутреннее
//             состояние структуры
//   |format| - имея заполненное внутреннее состояние, выполнить
//              форматирование для конкретного аргумента
template<>
struct fmt::formatter<Point>
{
  enum class Form
  {
    def,  // default form for {}
    fix,  // fixed format for {:f}
    exp   // exponential  for {:e}
  };
  Form form = Form::def;

  // отметьте здесь constexpr
  constexpr auto parse(format_parse_context& ctx)
  {
    // [ctx.begin(), ctx.end()) - подстрока для парсинга.
    //
    // В таком вызове:
    //
    //   fmt::format("{:f} - point of interest", point{1, 2});
    //
    // подстрока равна "f} - point of interest".
    //
    // Обязанность метода - разобрать строку формата до '}' и вернуть,
    // итератор, указывающий на '}', если не получилось - кинуть особое исключение.
    const char error_message_invalid_format[] =
        "invalid format for Point argument, expected {:f} or {:e} or {}";

    auto it = ctx.begin();
    const auto end = ctx.end();

    // случай "{}"
    if (it != end && *it == '}')
        // сразу возвращаем итератор на "}"
        return it;

    // считаем f или e из строки формата
    if (it != end)
    {
        if (*it == 'f')
            form = Form::fix;
        else if (*it == 'e')
            form = Form::exp;
        else
            throw format_error(error_message_invalid_format);

        ++it;
    }

    // убедимся, что в строке формата больше ничего нет,
    // и мы остановились именно на "}"
    if (it == end || *it != '}')
        throw format_error(error_message_invalid_format);

    // возвращаем итератор на "}"
    return it;
  }

  template <typename FormatContext>
  auto format(const Point& p, FormatContext& ctx)
  {
    const char* const fmt = form == Form::fix ? "({:.1f}, {:.1f}, {:.1f})" :
                            form == Form::exp ? "({:.1e}, {:.1e}, {:.1e})" :
                                                "({}, {}, {})";
    return format_to(ctx.out(), fmt, p.x, p.y, p.z);
  }
};
```

Использование:

```c++
int main()
{
    std::cout << fmt::format(" {0:f}\n {0:e}\n {0}", Point{3.f, 4.f, 5.f}) << std::endl;
}
```

Вывод:

```sh
 (3.0, 4.0, 5.0)
 (3.0e+00, 4.0e+00, 5.0e+00)
 (3.0, 4.0, 5.0)
```

<br />

Трюк для красивого вывода enum-ов:

```c++
enum class Color {
    red,
    green,
    blue
};

template <>
struct fmt::formatter<Color> : formatter<string_view>
{
  // метод |parse| отнаследован от форматера строк,
  // а значит наш форматер Color уже умеет понимать
  // все возможные способы отформатировать строку,
  // осталось только эту строку предоставить!
  
  template <typename FormatContext>
  auto format(Color c, FormatContext& ctx)
  {
    string_view name = "unknown";
    switch (c) {
    case color::red:   name = "red";   break;
    case color::green: name = "green"; break;
    case color::blue:  name = "blue";  break;
    }
    return formatter<string_view>::format(name, ctx);
  }
};

// usage:
fmt::print("{:>20}", Color::red);
```

<br />

###### несколько приятных утилит из fmtlib

shortcut для конвертации в строку через формат по умолчанию (замена для `std::format("{}", x)`):

```c++
std::string s1 = fmt::to_string(42);
std::string s2 = fmt::to_string(3.14);
```

`fmt::join`. Обратите внимание, что `fmt::join` принимает range произвольных типов:

```c++
std::vector<int> v = {1, 2, 3};
fmt::print("{}", fmt::join(v, ", "));  // Output: "1, 2, 3"
```

Буфер - аналог small vector

```c++
// fmt::memory_buffer out;
fmt::memory_buffer<char, 256> out;
format_to(out, "The answer is {}.", 42);
```

Форматирование дат

```c++
std::time_t t = std::time(nullptr);
fmt::print("The date is {:%Y-%m-%d}.", *std::localtime(&t));  // Prints "The date is 2016-04-29."
```

<br />

##### std::format (since C++20)

https://en.cppreference.com/w/cpp/utility/format

https://en.cppreference.com/w/cpp/utility/format/formatter

`fmtlib` оказался настолько хорош, что его доработали и перенесли в стандартную библиотеку С++, где назвали `std::format`. С С++20 можно пользоваться продвинутыми способами форматирования без сторонних библиотек.

<br />

##### сравнение библиотек

Скорость исполнения, [подробности эксперимента](https://github.com/fmtlib/fmt#speed-tests)

| Library           | Method        | Run Time, s 
|:------------------|:--------------|:-----------:
| libc              | printf        | 1.04        
| libc++            | std::ostream  | 3.05        
| {fmt} 6.1.1       | fmt::print    | 0.75        
| Boost Format 1.67 | boost::format | 7.24        
| Folly Format      | folly::format | 2.23        

Время компиляции, [подробности эксперимента](https://github.com/fmtlib/fmt#compile-time-and-code-bloat)

| Method        | Compile Time, s |
|:--------------|:---------------:|
| printf        | 2.6             |
| printf+string | 16.4            |
| iostreams     | 31.1            |
| {fmt}         | 19.0            |
| Boost Format  | 91.9            |
| Folly Format  | 115.7           |

Размер исполняемого файла, [подробности эксперимента](https://github.com/fmtlib/fmt#compile-time-and-code-bloat)

| Method        | Executable size, KiB | Stripped size, KiB |
|:--------------|:--------------------:|:------------------:|
| printf        | 29                   | 26                 |
| printf+string | 29                   | 26                 |
| iostreams     | 59                   | 55                 |
| {fmt}         | 37                   | 34                 |
| Boost Format  | 226                  | 203                |
| Folly Format  | 101                  | 88                 |

<br />

##### Резюме

* В обычном С++ приложении для форматирования строк используйте `std::format` с 20-го стандарта либо `fmtlib` до С++20.